In [2]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass
password = getpass.getpass()

········


In [8]:
connection_string = 'mysql+pymysql://root:' + password + '@127.0.0.1/sakila' # database_location/database_name
engine = create_engine(connection_string)

In [12]:
data_may = pd.read_sql_query('SELECT * FROM rentals_may', engine)
data_june = pd.read_sql_query('SELECT * FROM rentals_june', engine)

data_may.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1,2005-05-24 22:53:30,367,130,2005-05-26 22:04:30,1,2006-02-15 21:30:53
1,2,2005-05-24 22:54:33,1525,459,2005-05-28 19:40:33,1,2006-02-15 21:30:53
2,3,2005-05-24 23:03:39,1711,408,2005-06-01 22:12:39,1,2006-02-15 21:30:53
3,4,2005-05-24 23:04:41,2452,333,2005-06-03 01:43:41,2,2006-02-15 21:30:53
4,5,2005-05-24 23:05:21,2079,222,2005-06-02 04:33:21,1,2006-02-15 21:30:53


In [11]:
data_june.head()

,rental_id,rental_date,inventory_id,customer_id,return_date,staff_id,last_update
0,1158,2005-06-14 22:53:33,1632,416,2005-06-18 21:37:33,2,2006-02-15 21:30:53
1,1159,2005-06-14 22:55:13,4395,516,2005-06-17 02:11:13,1,2006-02-15 21:30:53
2,1160,2005-06-14 23:00:34,2795,239,2005-06-18 01:58:34,2,2006-02-15 21:30:53
3,1161,2005-06-14 23:07:08,1690,285,2005-06-21 17:12:08,1,2006-02-15 21:30:53
4,1162,2005-06-14 23:09:38,987,310,2005-06-23 22:00:38,1,2006-02-15 21:30:53


In [15]:
rentals_may = pd.read_sql_query('SELECT customer_id, COUNT(*) AS number_of_rentals_may FROM rentals_may GROUP BY customer_id', engine)
rentals_may

,customer_id,number_of_rentals_may
0,130,6
1,459,2
2,408,6
3,333,2
4,222,10
...,...,...
515,191,4
516,351,2
517,10,2
518,136,2


In [17]:
rentals_june = pd.read_sql_query('SELECT customer_id, COUNT(*) AS number_of_rentals_june FROM rentals_june GROUP BY customer_id', engine)
rentals_june

,customer_id,number_of_rentals_june
0,416,10
1,516,12
2,239,10
3,285,6
4,310,12
...,...,...
585,412,2
586,335,2
587,226,4
588,22,2


In [20]:
def compare_rentals(rentals_may, rentals_june):
    merged_data = pd.merge(rentals_may, rentals_june, on='customer_id', how='outer')
    merged_data['comparison'] = merged_data.apply(lambda row: 'More' if row['number_of_rentals_june'] > row['number_of_rentals_may']
                                                    else 'Less' if row['number_of_rentals_june'] < row['number_of_rentals_may']
                                                    else 'Equal', axis=1)
    return merged_data

comparison_data = compare_rentals(rentals_may, rentals_june)
comparison_data

,customer_id,number_of_rentals_may,number_of_rentals_june,comparison
0,130,6.0,12.0,More
1,459,2.0,14.0,More
2,408,6.0,6.0,Equal
3,333,2.0,8.0,More
4,222,10.0,4.0,Less
...,...,...,...,...
593,598,NaN,2.0,Equal
594,487,NaN,2.0,Equal
595,555,NaN,2.0,Equal
596,335,NaN,2.0,Equal
